In [1]:
!echo $PATH

/home/benjamin/anaconda3/bin:/home/benjamin/anaconda3/ensembl-vep:/home/benjamin/anaconda3/mummer-4.0.0/bin:/home/benjamin/anaconda3/bin:/usr/local/bioinfo/tcoffee11/bin:/home/benjamin/bin:/home/benjamin/anaconda3/ensembl-vep:/home/benjamin/anaconda3/mummer-4.0.0/bin:/home/benjamin/anaconda3/bin:/usr/local/bioinfo/tcoffee11/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin:/usr/local/bioinfo/a5/bin:/usr/local/bioinfo/aragorn1.2.38:/usr/local/bioinfo/barrnap-0.7/bin:/usr/local/bioinfo/ca/bin:/usr/local/bioinfo/canu/bin:/usr/local/bioinfo/cap3:/usr/local/bioinfo/hisat2:/usr/local/bioinfo/kraken-0.10.5-beta:/usr/local/bioinfo/nanopolish:/usr/local/bioinfo/NOVOPlasty:/usr/local/bioinfo/Organelle_PBA:/usr/local/bioinfo/PASApipeline/bin:/usr/local/bioinfo/phylip:/usr/local/bioinfo/pilon:/usr/local/bioinfo/Platypus_0.8.1:/usr/local/bioinfo/prokka/bin:/usr/local/bioinfo/prottest3:/usr/local/bioinfo/signalp-4.1:/usr/local/bioinfo/soap:/usr/local/bioinfo/SPAdes/bin:/usr/

In [2]:
%matplotlib inline
import pandas as pd
import os
import re
from pybedtools import BedTool
import numpy as np
import pybedtools
import re
import matplotlib.pyplot as plt
import json
import matplotlib
import subprocess
import numpy as np
import shutil
import seaborn as sns

ModuleNotFoundError: No module named 'pybedtools'

In [ ]:
#define your input folders
#define your input folders updated for haplotigs
EFFECTORP_FOLDER = '../../../genome_v03/annotation/functional/'
BUSCO_FOLDER = '../../../genome_v03/annotation/BUSCO/'
GFF_FOLDER = '../../../genome_v03/annotation/structural/'
GENOME_FOLDER = '../../genome/'
TE_FOLDER = '../../TE_analysis/'
EFFECTOR_FOLDER = '../../../genome_v03/effector_analysis'
PROTEIN_ANNO_FOLDER = '../../annotation/functional/'
AT_analysis = '../../AT_analysis'
OUT_FOLDER = os.path.join(AT_analysis, 'Dinucl')
TMP_FOLDER = os.path.join(OUT_FOLDER, 'difacount')
CONTROL_FOLDER = os.path.join(AT_analysis, 'control_samples')
OUT_FOLDER_FIG = '../../figures/'
TMP_FIG_PATH = '../../../genome_v03/tmp_figures'
genome = 'APSI_primary_v1'
REPET_genome = 'MR_P2_a0'

In [ ]:
if not os.path.exists(OUT_FOLDER):
    os.mkdir(OUT_FOLDER)

In [ ]:
if not os.path.exists(TMP_FOLDER):
    os.mkdir(TMP_FOLDER)

In [ ]:
#now get all the file names
genome_fn = os.path.join(GENOME_FOLDER, '%s.fa' %genome)
test_fn = os.path.join(TMP_FOLDER, 'test.fa')
!head {genome_fn} > {test_fn}

In [ ]:
!head {test_fn}

In [ ]:
def facount(in_fn, out_fn):
    cmd = 'faCount -dinuc %s > %s' % \
    (os.path.abspath(in_fn), os.path.abspath(out_fn))
    output = subprocess.run(cmd, shell=True)
    return output.returncode

In [9]:
def do_RIP_call(fac_fn):
    "Return pandas dataframe with caculations reading in the faCount outfile."
    faC_header = ['seqID','len','A','C','G','T','N','cpg','AA','AC','AG',
                 'AT','CA','CC','CG','CT','GA','GC','GG','GT','TA','TC','TG','TT']
    count_dict = {}
    for n_pattern in faC_header:
        count_dict[n_pattern] = []
    dtypes = {}
    for x in faC_header:
        dtypes[x] = 'int'
    dtypes['seqID'] = 'str'
    with open(fac_fn) as fh: 
        for line in fh:
            line = line.rstrip()
            split_line = re.split('\t', line)
            if not line.startswith('#'):
                #if len(split_line) > len(faC_header):
                  #  split_line = split_line[0:len(faC_header)+1]

                for n_pattern, value in zip(faC_header, split_line):
                    count_dict[n_pattern].append(value)
    df = pd.DataFrame.from_dict(count_dict)
    df = df.astype(dtypes)
    df['TpA/ApT'] = df['TA']/df['AT']
    df['CpA + TpG/ApC + GpT'] = (df['CA'] + df['TG'])/(df['AC']+df['GT'])
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df

In [10]:
def AT_content_df(bed_fn, contig_fn, intervals, merge=True):
    """Caculate the %AT for contigs and group %AT in specific intervals."""
    bed = BedTool(bed_fn)
    if merge == True:
        bed.sort().merge()
    #specify datatypes if possible
    tmp_fn = bed_fn.replace('.bed', '.AT.bed.tmp')
    dtype_dict = {0:str, 1:np.int32, 2:np.int32, 3:str, 4:str, 5:str, 6:float}
    bed.nucleotide_content(fi=contig_fn).saveas(tmp_fn)
    #bedtools nuc -fi {contig_fn} -bed {bed_fn} > {tmp_fn}
    
    if len(bed.to_dataframe().columns) == 6:
        dtype_dict = {0:str, 1:np.int32, 2:np.int32, 3:str, 4:str, 5:str, 6:float}
        df = pd.read_csv( tmp_fn, sep='\t',header=None,dtype=dtype_dict, usecols=[0,1,2,3,6], skiprows=1)
        df.rename(columns={6:'%AT'}, inplace=True)
    elif len(bed.to_dataframe().columns) == 3:
        dtype_dict = {0:str, 1:np.int32, 2:np.int32, 3:float}
        df = pd.read_csv( tmp_fn, sep='\t',header=None,dtype=dtype_dict, usecols=[0,1,2,3], skiprows=1)
        df.rename(columns={3:'%AT'}, inplace=True)
    df['%AT'] = df['%AT'].astype('float')
    df['W_Len'] = df[2] - df[1]
    
    df['Intervals'] = pd.cut(df['%AT'], intervals)
    grouped = df.groupby('Intervals')
    ###group by the edge
    ####get the length of the intervals and plot the sum for each interval
    ###edge
    
    
    sum_df = grouped.agg({'%AT': 'count', 'W_Len': 'sum'})
    sum_df['Interval_edge'] = intervals.right
    sum_df.fillna(0, inplace=True)
    sum_df['norm_count_AT'] = sum_df['%AT']/sum_df['%AT'].sum()
    sum_df['norm_W_Len'] = sum_df['W_Len']/sum_df['W_Len'].sum()
    os.remove(tmp_fn)
    return df, sum_df

In [11]:
def RM_subset(RM_fn, selection):
    """Pulls in the the RM gff and subsets it for the selected
    features entered as lis."""
    df = pd.read_csv(RM_fn, header = None, comment='#', sep = '\t')
    tmp_fn_list = []
    for select in selection:
        tmp_df = df[df[8].str.contains(select)]
        tmp_out_fn = RM_fn.replace('.gff', F'.{select}.gff')
        tmp_df.to_csv(tmp_out_fn, header = None, index = None, sep = '\t')
        tmp_fn_list.append(tmp_out_fn)
    return tmp_fn_list

SyntaxError: invalid syntax (<ipython-input-11-ce144e255aba>, line 8)

In [ ]:
def reverse_bed(genome_fn, gff_fn):
    """A bit of a hack to get the reverse of 
    a gff_fn"""
    !samtools faidx {genome_fn}
    tmp_fn = genome_fn + '.genome_file'
    !cut -f1,2 {genome_fn}.fai | sort -k 1,1 > {tmp_fn}
    tmp_out_fn = gff_fn.replace('.gff', '.reverse.nr.bed')
    pybedtools.BedTool(gff_fn).sort()\
    .merge().complement(g=tmp_fn).saveas(tmp_out_fn)
    return tmp_out_fn

In [ ]:
#rename some of the genome headers as the initial genome had different contig names as the recent genome
repet_genome_fn = os.path.join(AT_analysis, '%s.repet.fa' % genome)
with open(genome_fn, 'r') as g_fh:
    with open(repet_genome_fn, 'w') as rg_fh:
        for line in g_fh:
            line = line.rstrip('\n')
            if line.startswith(">"):
                line = line.replace('APSI_P0','ApsiP_S00')
                print(line, file = rg_fh)
            else:
                print(line, file = rg_fh)
#do some indexing of genome and generate the genome file
!samtools faidx {repet_genome_fn}
!cut -f1,2 {repet_genome_fn}.fai > {repet_genome_fn.replace('.fa', '.genome')}
repet_genome_index_fn = os.path.abspath(repet_genome_fn.replace('.fa', '.genome'))

In [ ]:
#generate the non-TE genome bedfile
repet_gff_fn = os.path.join(TE_FOLDER, 'APSI_primary_v1.MR_P2A_300Mb.REPET.gff3' )
repet_bed_reverse_fn = repet_gff_fn.replace('.gff3', '.reverse.nr.bed')
pybedtools.BedTool(repet_gff_fn).sort().merge().\
complement(g=repet_genome_index_fn).saveas(repet_bed_reverse_fn)

In [ ]:
#collect all the non-redundant bed files for the RNA and DNA transposons annotated in the myrtle rust genome
###consider not to use the nr bed but look at strand specific sequences####
###This can be done by subsetting the GFF3 dataframe by specific column###
###Thinking about this I don't think the strand matters as the revers complement of TA is TA###
repet_bed_flies = []
for folder in ['Retrotransposon', 'DNA_transposon']:
    for file in os.listdir(os.path.join(TE_FOLDER, folder)):
        if file.endswith('.nr.bed') and file.split('.')[3] == 'superfamily':
            repet_bed_flies.append(os.path.join(TE_FOLDER, folder, file))
repet_bed_flies.append(repet_bed_reverse_fn)


In [ ]:
#collect all the DNA sequences for the repet bedfiles to do the facount on them
###consider not to use the nr bed but look at strand specific sequences####
facount_fa_files = []
for tmp_bed_fn in repet_bed_flies:
    tmp_bed = BedTool(tmp_bed_fn)
    tmp_out_fn = os.path.join(TMP_FOLDER, os.path.basename(tmp_bed_fn)\
                              .replace('.bed', '.fa'))
    tmp_bed.sequence(fi=repet_genome_fn).save_seqs(tmp_out_fn)
    facount_fa_files.append(tmp_out_fn)

In [ ]:
#pulling in the AT files based on occulter_cut and saving out the files
#and saving out the regions that are higher and lower 61% AT
intervals = pd.interval_range(start=0.2, end=1.0, closed='right', periods=80)
oc_bed_fn = os.path.join(AT_analysis, 'regions.bed')

oc_df, oc_sum_df = AT_content_df(oc_bed_fn, genome_fn, intervals)
oc_greater_fn = os.path.join(AT_analysis, 'Greater_61%AT_regions.bed')
oc_smaller_fn = os.path.join(AT_analysis, 'Smaller_61%AT_regions.bed')

oc_df[oc_df['%AT'] > 0.61].iloc[:, 0:3].to_csv(oc_greater_fn, index=None, header=None, sep='\t')
oc_df[(oc_df['%AT'] < 0.61)|(oc_df['%AT'] == 0.61)].iloc[:, 0:3].to_csv(oc_smaller_fn, index=None, header=None, sep='\t')

In [ ]:
#collect some more bedfiles of interest and get the sequences for 
#facount
bed_files = []
bed_files.append(os.path.join(EFFECTOR_FOLDER, 'APSI_primary_v1.transcript.genes.bed'))
bed_files.append(oc_greater_fn)
bed_files.append(oc_smaller_fn)

for tmp_bed_fn in bed_files:
    tmp_bed = BedTool(tmp_bed_fn)
    tmp_out_fn = os.path.join(TMP_FOLDER, os.path.basename(tmp_bed_fn)\
                              .replace('.bed', '.fa'))
    tmp_bed.sequence(fi=genome_fn).save_seqs(tmp_out_fn)
    facount_fa_files.append(tmp_out_fn)

In [ ]:
#also add the genome file to the mix
tmp_repet_genome_fn = os.path.join(TMP_FOLDER, os.path.basename(repet_genome_fn))
shutil.copyfile(repet_genome_fn, tmp_repet_genome_fn)
facount_fa_files.append(tmp_repet_genome_fn)

In [ ]:
facount_fa_files

In [ ]:
#cacluate the dinucleodite frequencies for all the given fa files
#store the counts and sequence lenght in three dictionaries
facount_fac_files = []
#'TpA/ApT'
dicount_dict_S = {}
#'CpA + TpG/ApC + GpT'
dicount_dict_L = {}
#length
dicount_dict_length = {}
for file in facount_fa_files:
    tmp_out_fn = file.replace('.fa', '.fac')
    facount(file, tmp_out_fn)
    facount_fac_files.append(tmp_out_fn)
    tmp_df = do_RIP_call(tmp_out_fn)
    if 'superfamily' in file:
        name = os.path.basename(file).split('.')[4]
    elif 'REPET.reverse.nr.' in file:
        name = 'non_TE'
    elif 'transcript.genes' in file:
        name = 'genes'
    elif '%AT_regions' in file:
        name = os.path.basename(file).split('.')[0]
    elif 'APSI_primary_v1.repet.fa' in file:
        name = os.path.basename(file)[:-3]
    dicount_dict_S[name] = tmp_df['TpA/ApT'].values
    dicount_dict_L[name] = tmp_df['CpA + TpG/ApC + GpT'].values
    dicount_dict_length[name] = tmp_df['len'].values

In [ ]:
#reminder of how to filter things
dicount_dict_S['APSI_primary_v1.repet'][dicount_dict_length['APSI_primary_v1.repet'] > 100000]

In [ ]:
#'CpA + TpG/ApC + GpT: dicount_dict_L
for x in dicount_dict_L.keys():
    print(x, ': ', np.median(dicount_dict_L[x][(dicount_dict_length[x] > 1000)&(~np.isnan(dicount_dict_L[x]))][:-1]))

In [ ]:
#'TpA/ApT': dicount_dict_S increases due to RIP
for x in  dicount_dict_S.keys():
    print(x, ': ', np.median(dicount_dict_S[x][(dicount_dict_length[x] > 500)&(~np.isnan(dicount_dict_S[x]))][:-1]))

In [ ]:
list_of_interest = ['non_TE', 'genes', 'ClassI:LTR:Gypsy','ClassI:LTR:Copia', 'Greater_61%AT_regions', 'Smaller_61%AT_regions', 'APSI_primary_v1.repet']

In [ ]:
cutoff = 500
col = 4
rows = len(list_of_interest) // col
if len(list_of_interest) % col > 0:
    rows +=1
print(rows)

In [ ]:
#'TpA/ApT': dicount_dict_S increases due to RIP
fig, ax = plt.subplots(rows, col, figsize = (20, 10), sharey=True)
gene_mean = np.median(dicount_dict_S['genes']\
                      [(dicount_dict_length['genes'] > cutoff)&(~np.isnan(dicount_dict_S['genes']))][:-1])
non_TE_mean = np.median(dicount_dict_S['non_TE']\
                      [(dicount_dict_length['non_TE'] > cutoff)&(~np.isnan(dicount_dict_S['non_TE']))][:-1])
for n, key in enumerate(list_of_interest):
    r = n // col
    c = n % col
    ax[r,c].boxplot(dicount_dict_S[key][(dicount_dict_length[key] > cutoff)&(~np.isnan(dicount_dict_S[key]))][:-1])
    ax[r,c].set_title(key)
    ax[r,c].set_ylim(0,2)
    ax[r,c].axhline(gene_mean, c='r', ls='--')
    ax[r,c].axhline(non_TE_mean, c='g', ls='--')
    #ax[]

In [ ]:
#maybe do the rip analysis by family instead
#maybe consider doing reverse complements as well

In [ ]:
#now pull in the control files
GENOME_NAME_DICT = {}
GENOME_NAME_DICT['Blumeria graminis f. sp. hordei'] = 'Blumeria_graminis_f.sp.hordei_DH14__GCA_900239735.1_BGH_DH14_v4_genomic'
GENOME_NAME_DICT['Marssonina brunnea'] = 'Marssonina_brunnea_GCF_000298775.1_ASM29877v1_genomic'
GENOME_NAME_DICT['Rhynchosporium commune'] = 'Rhynchosporium_commune__GCA_900074885.1_version_1_genomic'
GENOME_NAME_DICT['Parauncinula polyspora'] = 'parauncinula_polyspora.scaffolds'

In [ ]:
#first pull in the genome files
#then pull in the gff files and subset them
GENOME_FN_DICT = {}
for key, value in GENOME_NAME_DICT.items():
    if key != 'Parauncinula polyspora':
        GENOME_FN_DICT[key] = os.path.join(CONTROL_FOLDER, value + '.fna')
    else:
        GENOME_FN_DICT[key] = os.path.join(CONTROL_FOLDER, value + '.fa')

In [ ]:
GFF_FN_DICT = {}
for key, value in GENOME_FN_DICT.items():
    GFF_FN_DICT[key] = value + '.out.gff'

In [ ]:
SUB_GFF_FN_DICT = {}
selection = ['Tad1', 'Gypsy', 'Copia']
for key, value in GFF_FN_DICT.items():
    SUB_GFF_FN_DICT[key] = RM_subset(value, selection)

In [ ]:
###Generate the non-TE GFF/bed which requires 
### the genome file as well
selection.append('non_TE')
for key, value in SUB_GFF_FN_DICT.items():
    tmp_fn = reverse_bed(GENOME_FN_DICT[key], GFF_FN_DICT[key])
    #print(value.append(tmp_fn))
    SUB_GFF_FN_DICT[key].append(tmp_fn)

In [ ]:
#next steps is to generate the whole counting
#and result dicts followed by plotting.
SUB_GFF_FN_DICT

In [ ]:
#get the fa and fac count done.
SUB_FA_FN_DICT = {}
SUB_FAC_FN_DICT = {}
for key, bed_files in SUB_GFF_FN_DICT.items():
    tmp_fa_list = []
    tmp_fac_list = []
    for tmp_bed_file in bed_files:
        #get the fa files from the bedfiles
        ending = os.path.basename(tmp_bed_file).split('.')[-1]
        tmp_fa_out_fn = os.path.join(TMP_FOLDER, os.path.basename(tmp_bed_file)\
                              .replace(ending, 'fa'))
        tmp_bed = BedTool(tmp_bed_file)
        tmp_bed.sequence(fi=GENOME_FN_DICT[key]).save_seqs(tmp_fa_out_fn)
        tmp_fa_list.append(tmp_fa_out_fn)
        #now do the fac count
        tmp_fac_out_fn = tmp_fa_out_fn.replace('.fa' , '.fac')
        facount(tmp_fa_out_fn, tmp_fac_out_fn)
        tmp_fac_list.append(tmp_fac_out_fn)
    SUB_FA_FN_DICT[key] = tmp_fa_list
    SUB_FAC_FN_DICT[key] = tmp_fac_list    


In [ ]:
##rename things to be name compatible with the upstream part ['Tad1', 'Gypsy', 'Copia', 'non_TE']
new_selection = ['Tad1', 'ClassI:LTR:Gypsy','ClassI:LTR:Copia', 'non_TE']

In [ ]:
SUB_S_DICT = {}
SUB_L_DICT = {}
SUB_length_DICT = {}

for key, fac_list in SUB_FAC_FN_DICT.items():
    #now get the different dictionaries
    #'TpA/ApT'
    tmp_S = {}
    #'CpA + TpG/ApC + GpT'
    tmp_L = {}
    #length
    tmp_length = {}
    for name, tmp_fac_fn in zip(new_selection, fac_list):
        print(name)
        
        tmp_df = do_RIP_call(tmp_fac_fn)
        #selection would be the keys
        tmp_S[name] = tmp_df['TpA/ApT'].values
        tmp_L[name] = tmp_df['CpA + TpG/ApC + GpT'].values
        tmp_length[name] = tmp_df['len'].values
    SUB_S_DICT[key] = tmp_S
    SUB_L_DICT[key] = tmp_L
    SUB_length_DICT[key] = tmp_length

In [ ]:
print('#TpA/ApT')
for key, tmp_dict in SUB_S_DICT.items():
    print(F'\n\n#####{key}####')
    for name in new_selection[1:]:
        tmp_x = np.median(tmp_dict[name][(SUB_length_DICT[key][name] > 500)&(~np.isnan(tmp_dict[name]))][:-1])
        print(F'This is the median for {name}: {tmp_x}')


In [ ]:
print('CpA + TpG/ApC + GpT')
for key, tmp_dict in SUB_L_DICT.items():
    print(F'\n\n#####{key}####')
    for name in new_selection[1:]:
        tmp_x = np.median(tmp_dict[name][(SUB_length_DICT[key][name] > 500)&(~np.isnan(tmp_dict[name]))][:-1])
        print(F'This is the median for {name}: {tmp_x}')


In [ ]:
#now the plotting
cutoff = 500
col = 5
rows = 5 // col
if 5 % col > 0:
    rows +=1
print(rows)

In [ ]:
cutoff = 0


In [ ]:
ordered_keys = [ 'Marssonina brunnea', 'Rhynchosporium commune', 'Parauncinula polyspora', 'Blumeria graminis f. sp. hordei']

In [ ]:
print('#TpA/ApT')
sns.set_style("whitegrid")
sns.set_context("poster")
boxprops = dict(linewidth=3, color='b')
medianprops = dict(linestyle='-', linewidth=2.5, color='firebrick')
whiskerprops = dict(linewidth=1.5, color='b')

fig, ax = plt.subplots(rows, col, figsize = (20, 6), sharey=True)
new_selection = ['non_TE','ClassI:LTR:Gypsy','ClassI:LTR:Copia']

#do AP first
non_TE_mean = np.median(dicount_dict_S['non_TE']\
                      [(dicount_dict_length['non_TE'] > cutoff)&(~np.isnan(dicount_dict_S['non_TE']))][:-1])
tmp_data = []
for n, key in enumerate(new_selection):
    tmp_val = dicount_dict_S[key][(dicount_dict_length[key] > cutoff)&(~np.isnan(dicount_dict_S[key]))][:-1]
    tmp_data.append(tmp_val)

tmp_box = ax[0].boxplot(tmp_data, vert=True,  # vertical box alignment
                         patch_artist=False,
             widths=0.8,
            boxprops=boxprops,
            medianprops=medianprops,
            whiskerprops = whiskerprops)
ax[0].axhline(non_TE_mean, c='black', ls='--', linewidth = 4)
ax[0].set_title('Austropuccinia psidii', pad = 20)
ax[0].set_xticklabels(new_selection, rotation = 70)
ax[0].set_ylim(-0.1, 4)
#tmp_box = 

count = 1

for key in ordered_keys:
    tmp_non_TE_mean = np.median(SUB_S_DICT[key]['non_TE'][(SUB_length_DICT[key]['non_TE'] > cutoff)\
                                               &(~np.isnan(SUB_S_DICT[key]['non_TE']))][:-1])
    tmp_data = []
    for name in new_selection:
        tmp_val  = SUB_S_DICT[key][name][(SUB_length_DICT[key][name] > cutoff)\
                                               &(~np.isnan(SUB_S_DICT[key][name]))][:-1]
        tmp_data.append(tmp_val)
    ax[count].boxplot(tmp_data, vert=True,  # vertical box alignment
                         patch_artist=False,
             widths=0.8,
            boxprops=boxprops,
            medianprops=medianprops,
            whiskerprops = whiskerprops)
    

    ax[count].axhline(tmp_non_TE_mean, c='black', ls='--', linewidth = 4)
    ax[count].set_title(key, pad= (20 + 30 * (count%2)))
    ax[count].set_xticklabels(new_selection, rotation = 70)
    count += 1
out_fn = os.path.join(OUT_FOLDER_FIG, 'TpA_etal_ratio.png')
fig.tight_layout()
fig.savefig(out_fn, dpi=600)

In [ ]:
print('CpA + TpG/ApC + GpT')
sns.set_style("whitegrid")
sns.set_context("poster")
boxprops = dict(linewidth=3, color='b')
medianprops = dict(linestyle='-', linewidth=2.5, color='firebrick')
whiskerprops = dict(linewidth=1.5, color='b')

fig, ax = plt.subplots(rows, col, figsize = (20, 6), sharey=True)
new_selection = ['non_TE','ClassI:LTR:Gypsy','ClassI:LTR:Copia']

#do AP first
non_TE_mean = np.median(dicount_dict_L['non_TE']\
                      [(dicount_dict_length['non_TE'] > cutoff)&(~np.isnan(dicount_dict_L['non_TE']))][:-1])
tmp_data = []
for n, key in enumerate(new_selection):
    tmp_val = dicount_dict_L[key][(dicount_dict_length[key] > cutoff)&(~np.isnan(dicount_dict_L[key]))][:-1]
    tmp_data.append(tmp_val)

tmp_box = ax[0].boxplot(tmp_data, vert=True,  # vertical box alignment
                         patch_artist=False,
             widths=0.8,
            boxprops=boxprops,
            medianprops=medianprops,
            whiskerprops = whiskerprops)
ax[0].axhline(non_TE_mean, c='black', ls='--', linewidth = 4)
ax[0].set_title('Austropuccinia psidii', pad = 20)
ax[0].set_xticklabels(new_selection, rotation = 70)
ax[0].set_ylim(-0.1, 4)
#tmp_box = 

count = 1

for key in ordered_keys:
    tmp_non_TE_mean = np.median(SUB_L_DICT[key]['non_TE'][(SUB_length_DICT[key]['non_TE'] > cutoff)\
                                               &(~np.isnan(SUB_L_DICT[key]['non_TE']))][:-1])
    tmp_data = []
    for name in new_selection:
        tmp_val  = SUB_L_DICT[key][name][(SUB_length_DICT[key][name] > cutoff)\
                                               &(~np.isnan(SUB_L_DICT[key][name]))][:-1]
        tmp_data.append(tmp_val)
    ax[count].boxplot(tmp_data, vert=True,  # vertical box alignment
                         patch_artist=False,
             widths=0.8,
            boxprops=boxprops,
            medianprops=medianprops,
            whiskerprops = whiskerprops)
    

    ax[count].axhline(tmp_non_TE_mean, c='black', ls='--', linewidth = 4)
    ax[count].set_title(key, pad= (20 + 30 * (count%2)))
    ax[count].set_xticklabels(new_selection, rotation = 70)
    count += 1
out_fn = os.path.join(OUT_FOLDER_FIG, 'CpA_etal_ratio.png')
fig.tight_layout()
fig.savefig(out_fn, dpi=600)

In [ ]:
/home/benjamin/myrtle_rust/genome_v03/scripts/notebooks
